In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer # tri druhy skalovani
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split # pro rozdeleni  dat
from sklearn.model_selection import KFold # pouziti cross validace
from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

df_final = pd.read_csv('final_dataset2.csv')

seed = 420
y = preprocessing.LabelEncoder().fit_transform(df_final['art_annotation'])

X = df_final
X = X.drop('art_annotation',axis = 1)
X = X.drop('icp_annotation',axis = 1)
X = X.drop('StartTime',axis = 1)
X = X.drop('EndTime',axis = 1)
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)


pipe_mm_rf200 = Pipeline([('scaler', MinMaxScaler()),('classifier', RandomForestClassifier(n_estimators=200))])
pipe_mm_rf500 = Pipeline([('scaler', MinMaxScaler()),('classifier', RandomForestClassifier(n_estimators=500))])
pipe_mm_knn5 = Pipeline([('scaler', MinMaxScaler()),('classifier', KNeighborsClassifier(n_neighbors=5))])
pipe_mm_knn7 = Pipeline([('scaler', MinMaxScaler()),('classifier', KNeighborsClassifier(n_neighbors=7))])
pipe_mm_knn9 = Pipeline([('scaler', MinMaxScaler()),('classifier', KNeighborsClassifier(n_neighbors=9))])
#pipe_mm_xcb = Pipeline([('scaler', MinMaxScaler()),('classifier', xgb.XGBClassifier())]) #n_estimators=500, max_depth=16) choose hyperparameters correctly?
#pipe_mm_cat = Pipeline([('scaler', MinMaxScaler()),('classifier', cb.CatBoostClassifier(random_state=seed, verbose = 0))])
pipe_sc_svc = Pipeline([('scaler', StandardScaler()),('classifier', SVC(gamma='auto', cache_size=4000, C=0.8))])

pipes = {"pipe_sc_svc":pipe_sc_svc}


# pro kazdou rouru si budeme drzet vysledky
results = { pipe_name: [] for pipe_name in pipes.keys()}
# rozdelime si data na trenovaci, ktere budeme delit dale, a testovaci, na kterych pak ukazeme chovani
X_tr, X_test, y_tr, y_test = train_test_split(X,y, test_size=0.2,random_state=seed) # testovaci data zatim nepouzijeme
# trenovaci mnozinu budeme delit dale na 5 podmnozin
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(X_tr, y_tr): # vraci dvojici poli testovacich a trenovacich indexu
  # rozdel si data na trenovaci a na data, na kterych bude ohodnocen klasifikator
  X_fold_tr = X_tr.values[train_index]
  y_fold_tr = y_tr[train_index]
  X_fold_test = X_tr.values[test_index]
  y_fold_test = y_tr[test_index]
  for k, pipe in pipes.items(): # pro kazdou pipe, delej
    pipe.fit(X_fold_tr, y_fold_tr) # nauc
    results[k].append(pipe.score(X_fold_test,y_fold_test)) # uloz si accuracy
    print(train_index,k)
#udelej si dataframe pro zhodnoceni
results = pd.DataFrame(data = results)
print(results) # tiskni za jednoltive foldy uspesnost klasifikatoru
print(results.mean()) # tiskni prumer

f1_results = {}

# Add the classification report and F1-score
for k, pipe in pipes.items():
    pipe.fit(X_tr, y_tr)
    y_pred_test = pipe.predict(X_test)
    f1 = f1_score(y_test, y_pred_test)  # You can change 'weighted' based on your requirement
    f1_results[k] = f1  # Store the F1-score
    
    print(f"Classification Report for {k}")
    print(classification_report(y_test, y_pred_test))
    print(f"F1-Score for {k}: {f1}")

# Convert F1-score results to DataFrame for better visualization
f1_results_df = pd.DataFrame(list(f1_results.items()), columns=['Pipeline', 'F1-Score'])
print(f1_results_df)

[   0    1    2 ... 6405 6406 6408] pipe_sc_svc
[   0    1    2 ... 6405 6406 6407] pipe_sc_svc
[   2    3    4 ... 6406 6407 6408] pipe_sc_svc
[   0    1    2 ... 6404 6407 6408] pipe_sc_svc
[   0    1    4 ... 6406 6407 6408] pipe_sc_svc
   pipe_sc_svc
0     0.910296
1     0.919657
2     0.914197
3     0.919657
4     0.918813
pipe_sc_svc    0.916524
dtype: float64
Classification Report for pipe_sc_svc
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      1479
           1       1.00      0.03      0.06       124

    accuracy                           0.93      1603
   macro avg       0.96      0.52      0.51      1603
weighted avg       0.93      0.93      0.89      1603

F1-Score for pipe_sc_svc: 0.0625
      Pipeline  F1-Score
0  pipe_sc_svc    0.0625


In [7]:
from sklearn.ensemble import IsolationForest

# Add Isolation Forest to your existing pipelines
pipe_mm_isoforest = Pipeline([('scaler', MinMaxScaler()),('classifier', IsolationForest(contamination=0.01))])

# Add the new pipeline to your existing list
pipes = {"pipe_sc_svc": pipe_sc_svc, "pipe_mm_isoforest": pipe_mm_isoforest}
# pro kazdou rouru si budeme drzet vysledky
results = { pipe_name: [] for pipe_name in pipes.keys()}
# rozdelime si data na trenovaci, ktere budeme delit dale, a testovaci, na kterych pak ukazeme chovani
X_tr, X_test, y_tr, y_test = train_test_split(X,y, test_size=0.2,random_state=seed) # testovaci data zatim nepouzijeme
# trenovaci mnozinu budeme delit dale na 5 podmnozin
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(X_tr, y_tr):
    X_fold_tr = X_tr.values[train_index]
    y_fold_tr = y_tr[train_index]
    X_fold_test = X_tr.values[test_index]
    y_fold_test = y_tr[test_index]
    
    for k, pipe in pipes.items():
        pipe.fit(X_fold_tr, y_fold_tr)
        
        if isinstance(pipe.named_steps['classifier'], IsolationForest):
            y_pred = pipe.predict(X_fold_test)
            y_pred = [1 if x == -1 else 0 for x in y_pred]  # Convert -1 to 1 for anomaly, and 1 to 0 for normal
            acc = sum(y_fold_test == y_pred) / len(y_fold_test)
            results[k].append(acc)
        else:
            results[k].append(pipe.score(X_fold_test, y_fold_test))
        
        print(train_index, k)

#udelej si dataframe pro zhodnoceni
results = pd.DataFrame(data = results)
print(results) # tiskni za jednoltive foldy uspesnost klasifikatoru
print(results.mean()) # tiskni prumer

f1_results = {}

# Add the classification report and F1-score
for k, pipe in pipes.items():
    pipe.fit(X_tr, y_tr)
    y_pred_test = pipe.predict(X_test)
    f1 = f1_score(y_test, y_pred_test)  # You can change 'weighted' based on your requirement
    f1_results[k] = f1  # Store the F1-score
    
    print(f"Classification Report for {k}")
    print(classification_report(y_test, y_pred_test))
    print(f"F1-Score for {k}: {f1}")

# Convert F1-score results to DataFrame for better visualization
f1_results_df = pd.DataFrame(list(f1_results.items()), columns=['Pipeline', 'F1-Score'])
print(f1_results_df)

[   0    2    3 ... 6406 6407 6408] pipe_sc_svc
[   0    2    3 ... 6406 6407 6408] pipe_mm_isoforest
[   0    1    2 ... 6406 6407 6408] pipe_sc_svc
[   0    1    2 ... 6406 6407 6408] pipe_mm_isoforest
[   0    1    2 ... 6405 6406 6407] pipe_sc_svc
[   0    1    2 ... 6405 6406 6407] pipe_mm_isoforest
[   0    1    3 ... 6405 6406 6408] pipe_sc_svc
[   0    1    3 ... 6405 6406 6408] pipe_mm_isoforest
[   1    2    4 ... 6405 6407 6408] pipe_sc_svc
[   1    2    4 ... 6405 6407 6408] pipe_mm_isoforest
   pipe_sc_svc  pipe_mm_isoforest
0     0.903276           0.916537
1     0.917317           0.923557
2     0.913417           0.913417
3     0.910296           0.915757
4     0.932865           0.928962
pipe_sc_svc          0.915434
pipe_mm_isoforest    0.919646
dtype: float64
Classification Report for pipe_sc_svc
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      1479
           1       1.00      0.03      0.06       124

    accur

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].